In [13]:
import numpy as np
import pandas as pd

In [14]:
#Load dataset
dfDataChampionship            =  pd.read_csv("campeonato-brasileiro-full.csv", delimiter=",")

#To String
dfDataChampionship            =  dfDataChampionship.applymap(str);  

#Create new column Placar
dfDataChampionship["Placar"]  =  dfDataChampionship["Clube 1 Gols"].map(str) + "x" + dfDataChampionship["Clube 2 Gols"]

#Show data
dfDataChampionship.head()

,Horário,Dia,Data,Clube 1,Clube 2,Vencedor,Rodada,Arena,Clube 1 Gols,Clube 2 Gols,Clube 1 Estado,Clube 2 Estado,Estado Clube Vencedor,Placar
0,16h00,Sábado,2000-07-29,Fluminense,Bahia,Fluminense,1ª Rodada,Maracanã,2,0,RJ,BA,RJ,2x0
1,16h00,Sábado,2000-07-29,Vasco,Sport,Sport,1ª Rodada,São Januário,0,2,RJ,PE,PE,0x2
2,16h00,Sábado,2000-07-29,Vitória,Palmeiras,Vitória,1ª Rodada,Barradão,4,1,ES,SP,ES,4x1
3,17h00,Domingo,2000-07-30,Botafogo-RJ,Atlético-MG,-,1ª Rodada,Caio Martins,0,0,RJ,MG,Empate,0x0
4,18h30,Domingo,2000-07-30,Juventude,Flamengo,-,1ª Rodada,Alfredo Jaconi,1,1,RS,RJ,Empate,1x1


In [15]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);

In [16]:
filter1  =  dfDataChampionship['Data']>='2017-01-01'
filter2  =  dfDataChampionship['Data']<='2017-12-31'
brasilianLeague2017 = dfDataChampionship[filter1 & filter2]
brasilianLeague2017.head()

,Horário,Dia,Data,Clube 1,Clube 2,Vencedor,Rodada,Arena,Clube 1 Gols,Clube 2 Gols,Clube 1 Estado,Clube 2 Estado,Estado Clube Vencedor,Placar
6799,16:00,Sábado,2017-05-13,Flamengo,Atlético-MG,-,1ª Rodada,Maracanã,1,1,RJ,MG,Empate,1x1
6800,19:00,Sábado,2017-05-13,Corinthians,Chapecoense,-,1ª Rodada,Arena Corinthians,1,1,SP,SC,Empate,1x1
6801,16:00,Domingo,2017-05-14,Avaí,Vitória,-,1ª Rodada,Ressacada,0,0,SC,ES,Empate,0x0
6802,16:00,Domingo,2017-05-14,Bahia,Athlético-PR,Bahia,1ª Rodada,Fonte Nova,6,2,BH,PR,BA,6x2
6803,16:00,Domingo,2017-05-14,Cruzeiro,São Paulo,Cruzeiro,1ª Rodada,Mineirão,1,0,MG,SP,MG,1x0


In [17]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([brasilianLeague2017['Clube 1'].str.lower(), brasilianLeague2017['Clube 2'].str.lower()], axis=1, keys=['Clubes'])


In [18]:
#Obtem todos os clubes 
cb  =  pd.Series(clubs['Clubes'].unique(), name="Clubes")
cb  =  cb.to_frame()
cb

,Clubes
0,flamengo
1,corinthians
2,avaí
3,bahia
4,cruzeiro
5,fluminense
6,grêmio
7,palmeiras
8,ponte preta
9,coritiba


In [19]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [20]:

dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = 0

def ensureUtf(s, encoding='utf8'):
  if type(s) == bytes:
    return s.decode(encoding, 'ignore')
  else:
    return s

for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    c1  =  ensureUtf( c1 )
    c1  =  c1.strip()
    
    pg       =  getPoints(brasilianLeague2017, c1 )
    j        =  getNumberDepartures(brasilianLeague2017, c1 )
    v        =  getNumberWinner(brasilianLeague2017, c1 )
    e        =  getDraw(brasilianLeague2017, c1 )
    d        =  getDefeats(brasilianLeague2017, c1 )
    gp, gc   =  getGP(brasilianLeague2017, c1 )
    
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc

dfTable['Clubes'] = dfTable['Clubes'].str.capitalize() 
dfTable  =  dfTable.sort_values(by=['PG', 'J', 'V', 'E', 'D', 'GP'], ascending=False)
dfTable  =  dfTable.reset_index(drop=True)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,Corinthians,72,38,21,9,8,50,30,20
1,Palmeiras,63,38,19,6,13,61,45,16
2,Santos,63,38,17,12,9,42,32,10
3,Grêmio,62,38,18,8,12,55,36,19
4,Cruzeiro,57,38,15,12,11,47,39,8
5,Flamengo,56,38,15,11,12,49,38,11
6,Vasco,56,38,15,11,12,40,47,-7
7,Chapecoense,54,38,15,9,14,47,49,-2
8,Atlético-mg,54,38,14,12,12,52,49,3
9,Botafogo-rj,53,38,14,11,13,45,42,3
